In [2]:
import pandas as pd
import numpy as np
import re
import os
import sys

DIR = os.getenv('KEA_BASE_DIR')
sys.path.append(DIR)

from cleaning import opencity13_const_corrector, opencity13_df_cleaner

In [3]:
df = pd.read_csv(r"2013DetailedResults.csv",index_col='_id')
df.head(5)

,Constituency No,Constituency Name,District Name,Winning Candidate,Sex,Age,Party,Constituency Type,Total Candidates,Total Electors,Valid Votes,Candidate Votes,Turnout %,Candidate Vote Share %,Winning Margin %,Deposit Lost ?
_id,,,,,,,,,,,,,,,,
1,1,NIPPANI,BELGAUM,JOLLE SHASHIKALA ANNASAHEB (Winner),F,43,BJP,General,12,189696,152690,81860,80.49,53.61,12.22,No
2,1,NIPPANI,BELGAUM,JITENDRA SUBHASH NERLE,M,38,IND,General,12,189696,152690,192,80.49,0.13,0.04,Yes
3,1,NIPPANI,BELGAUM,PATHAN NIYAZ,M,26,KJP,General,12,189696,152690,691,80.49,0.45,0.28,Yes
4,1,NIPPANI,BELGAUM,KAKASO PANDURANG PATIL (Runner Up),M,57,INC,General,12,189696,152690,63198,80.49,41.39,40.23,No
5,1,NIPPANI,BELGAUM,DR. CHANDRAKANT BASHETTI KURABETTI,M,63,IND,General,12,189696,152690,1545,80.49,1.01,0.20,Yes


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2948 entries, 1 to 2948
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Constituency No         2948 non-null   int64  
 1   Constituency Name       2948 non-null   object 
 2   District Name           2948 non-null   object 
 3   Winning Candidate       2948 non-null   object 
 4   Sex                     2948 non-null   object 
 5   Age                     2948 non-null   int64  
 6   Party                   2948 non-null   object 
 7   Constituency Type       2948 non-null   object 
 8   Total Candidates        2948 non-null   int64  
 9   Total Electors          2948 non-null   int64  
 10  Valid Votes             2948 non-null   int64  
 11  Candidate Votes         2948 non-null   int64  
 12  Turnout %               2948 non-null   float64
 13  Candidate Vote Share %  2948 non-null   float64
 14  Winning Margin %        2948 non-null   

In [13]:
# Extracting the constituency number and corresponding constituency name for further use.
num_df = df[['Constituency No','Constituency Name']]
num_df['Constituency Name'] = num_df['Constituency Name'].apply(opencity13_const_corrector)
num_df['Constituency No'] = num_df['Constituency No'].astype('Int32')
num_df.drop_duplicates(inplace=True)
num_df.set_index('Constituency No',inplace=True)
num_df.to_json('const_num.json')

C:\Users\bhagy\AppData\Local\Temp\ipykernel_26016\1885564102.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  num_df['Constituency Name'] = num_df['Constituency Name'].apply(opencity13_const_corrector)
C:\Users\bhagy\AppData\Local\Temp\ipykernel_26016\1885564102.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  num_df['Constituency No'] = num_df['Constituency No'].astype('Int32')
C:\Users\bhagy\AppData\Local\Temp\ipykernel_26016\1885564102.py:5: SettingWithCopyWarning: 
A value is trying to be set on a

In [78]:
# Cleaning the dataset
df = opencity13_df_cleaner(df)

# Correcting the party names.
df.party = df.party.apply(lambda x : 'BSRC' if x=='BSRCP' else x)

In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2948 entries, 1 to 2948
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   constituency                 2948 non-null   object 
 1   district                     2948 non-null   object 
 2   name                         2948 non-null   object 
 3   gender                       2948 non-null   object 
 4   age                          2948 non-null   int64  
 5   party                        2948 non-null   object 
 6   const_category               2948 non-null   object 
 7   total_electors               2948 non-null   int64  
 8   total_const_votes            2948 non-null   int64  
 9   votes                        2948 non-null   int64  
 10  candidate_voteshare_percent  2948 non-null   float64
dtypes: float64(1), int64(4), object(6)
memory usage: 276.4+ KB


In [80]:
# Converting to a csv file.
df.to_csv('candidates2013OpenCity.csv')

In [81]:
# Filtering the winners for each constituency 
def const_maker(df):
    return df.drop(['_id','constituency'],axis='columns').sort_values('votes',ascending=False).head(1).squeeze()

const_opencity13 = df.reset_index().groupby('constituency').apply(const_maker)
const_opencity13.head(5)

In [83]:
const_myneta13 = pd.read_csv('winners2013MyNeta.csv',index_col=0)
const_myneta13.head(5)

,constituency,name,party,age,cases,assets,liabilities,education,district,self_profession,spouse_profession
0,Badami,Chimmanakatti Balappa Bhimappa,INC,62,NaN,NaN,NaN,NaN,Bagalkot,NaN,NaN
1,Dasarahalli,S Muniraju,BJP,55,NaN,NaN,NaN,NaN,Bangalore Urban,NaN,NaN
2,Gurmitkal,Baburao Chinchanasoor,INC,62,NaN,NaN,NaN,NaN,Yadgir,NaN,NaN
3,Heggadadevankote,Chikkamadu S,JD(S),62,NaN,NaN,NaN,NaN,Mysore,NaN,NaN
4,Jagalur,H.P.Rajesh,INC,47,NaN,NaN,NaN,NaN,Davangere,NaN,NaN


In [86]:
# Renaming the winning candidates' names in this (votes dataset) dataset from the MyNeta dataset

temp_dic = {const_myneta13.loc[index,'constituency']:const_myneta13.loc[index,'name'] for index in const_myneta13.index}

for const in const_opencity13.index:
    const_opencity13.loc[const,'name'] = temp_dic[const]

const_opencity13.head(5)

,district,name,gender,age,party,const_category,total_electors,total_const_votes,votes,candidate_voteshare_percent
constituency,,,,,,,,,,
Afzalpur,Gulbarga,Malikayya Venkayya Guttedar,M,57,INC,GEN,190336,128606,38093,29.62
Aland,Gulbarga,B.R.Patil,M,63,KJP,GEN,192986,132385,67085,50.67
Anekal,Bangalore Urban,Shivanna.B,M,46,INC,SC,270767,186461,105464,56.56
Arabhavi,Belgaum,Balachandra Laxmanrao Jarkiholi,M,47,BJP,GEN,196454,145099,99283,68.42
Arkalgud,Hassan,Manju A,M,55,INC,GEN,197950,170262,61369,36.04


In [87]:
# Resetting the index back to default
const_opencity13 = const_opencity13.reset_index()

In [88]:
# Converting to a csv file.
const_opencity13.to_csv('constituency2013OpenCity.csv')